# LLM as Helpcenter agent

can also break down the wrapper for different model temperature

In [1]:
import duckdb
import openai
import pandas as pd
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders.duckdb_loader import DuckDBLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.agents import Tool

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model_name='gpt-4', temperature=0)
con = duckdb.connect('llm_demo.db')
try:
    con.sql("SELECT * from hc_articles limit 1")
except duckdb.CatalogException:
    hc_articles = pd.read_csv('../data/active_hc_articles.csv', usecols=['id','html_url','title','body'])
    con.sql("DROP TABLE IF EXISTS hc_articles")
    con.sql("CREATE TABLE hc_articles as select * from hc_articles")

loader = DuckDBLoader("SELECT * FROM hc_articles", database="llm_demo.db")
index = VectorstoreIndexCreator().from_loaders([loader])
tools = [
    Tool(
        name="Help center",
        func=index.query,
        description="useful for helping the user"
    )
]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm_chain = initialize_agent(tools=tools, llm=llm, agent="chat-conversational-react-description", memory=memory, verbose=True)

Using embedded DuckDB without persistence: data will be transient


In [2]:
print("bot: Hi, I am Wealthsimple's trusted AI agent! How can I help?\n")
while True:
    res = llm_chain.run(input=input("user: "))
    print(f"bot: {res}\n")

# explain to me the difference between tfsa and rrsp
# where do i check how much contribution room i have for tfsa?
# i want to DRS my shares.
# what's the fee for foreign transactions on your cash card
# is my funds safe with wealthsimple?
# i want to assign my beneficiary for RESP
# what is wealthsimple invest? how is it different from trade?

bot: Hi, I am Wealthsimple's trusted AI agent! How can I help?



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The main difference between a TFSA (Tax-Free Savings Account) and an RRSP (Registered Retirement Savings Plan) lies in their tax treatment and purpose. A TFSA allows you to save money and earn interest tax-free, while an RRSP is designed for retirement savings and provides tax deductions on contributions. With a TFSA, you can withdraw funds at any time without tax penalties, whereas with an RRSP, withdrawals are taxed as income and may be subject to penalties if withdrawn before retirement age. Both accounts have annual contribution limits and are useful for different financial goals."
}
```

> Finished chain.
bot: The main difference between a TFSA (Tax-Free Savings Account) and an RRSP (Registered Retirement Savings Plan) lies in their tax treatment and purpose. A TFSA allows you to save money and earn interest tax-free, 

KeyboardInterrupt: 